In [23]:
from typing import List, Tuple, Union
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import InputLayer, LSTM, GRU, Dense, Flatten, Conv1D
import glob
import os
from scipy.io.wavfile import read
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from numpy import ndarray
import midiDriver
import audioRecorder


def esr(signal_a, signal_b) -> float:
    '''Returns the Error-to-Signal Ratio.

    Keyword arguments:
    signal_a -- the groundtruth signal
    signal_b -- the predicted signal
    '''
    power = 2.0
    numerator = np.sum(np.power(np.subtract(signal_a, signal_b), power))
    denominator = np.sum(np.power(signal_a, power))
    return np.divide(numerator, denominator)
    

def normalize(array: Union[List,ndarray], scale_max: int=1, scale_min: int=0) -> List:
    '''Returns a normalized array.
    
    Keyword arguments:
    array -- array to normalize
    scale_max -- maximum value to scale between
    scale_min -- minimum value to scale between

    Source: https://www.geeksforgeeks.org/how-to-normalize-an-array-in-numpy-in-python/
    '''
    scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
    return scaler.fit_transform(array)


def partition_dataset(data_path: str='data/simple_dataset/01/*', train_perc: float=0.8) -> Tuple[List, List, List, List]: 
    '''Partition into train, train_labels & test, test_labels datasets.

    Keyword arguments:
    data_path -- where the data lives
    '''
    assert train_perc < 1, 'train_perc must be less than 1'
    data_paths = glob.glob(data_path)
    split_idx = int(0.8 * len(data_paths))
    train_paths, test_paths = data_paths[:split_idx], data_paths[split_idx:]
    train_data, test_data = [], []
    train_labels, test_labels = [], []
    for file in train_paths:
        _, data = read(file)
        train_data.append(data)
        train_labels.append(os.path.basename(file).split('.')[0])
    for file in test_paths:
        _, data = read(file)
        test_data.append(data)
        test_labels.append(os.path.basename(file).split('.')[0])
    return train_data, train_labels, test_data, test_labels


def make_model(input_shape: Tuple[int, int], num_output_nodes: int=3) -> Sequential:
    '''Return a model.
    
    Keyword arguments:
    num_output_nodes -- number of nodes on the output layer
    '''
    model = Sequential([
        InputLayer(input_shape=input_shape),
        # units = 8 is a value taken from https://arxiv.org/pdf/2009.02833.pdf
        GRU(units=8, return_sequences=True), 
        Dense(num_output_nodes)
    ])
    return model

In [2]:
# partition dataset
train_data_raw, train_labels_raw, test_data_raw, test_labels_raw = partition_dataset()

In [3]:
# normalize datasets
train_data = []
for i, d in enumerate(train_data_raw):
    if i % (len(train_data_raw) * 0.1) == 0:
        print(f'{i/len(train_data_raw):.2f}% train data normalization complete')
    train_data.append(normalize(d))
test_data = []
for i, d in enumerate(test_data_raw):
    if i % (len(test_data_raw) * 0.1) == 0:
        print(f'{i/len(test_data_raw):.2f}% test data normalization complete')
    test_data.append(normalize(d))

0.00% train data normalization complete
0.10% train data normalization complete
0.20% train data normalization complete
0.30% train data normalization complete
0.40% train data normalization complete
0.50% train data normalization complete
0.60% train data normalization complete
0.70% train data normalization complete
0.80% train data normalization complete
0.90% train data normalization complete
0.00% test data normalization complete
0.10% test data normalization complete
0.20% test data normalization complete
0.30% test data normalization complete
0.40% test data normalization complete
0.50% test data normalization complete
0.60% test data normalization complete
0.70% test data normalization complete
0.80% test data normalization complete
0.90% test data normalization complete


In [4]:
# parse labels
train_labels = []
for l in train_labels_raw:
    train_labels.append([param_pair[1] for param_pair in eval(l)])

In [5]:
# reshape features & labels 
train_data = np.asarray(train_data)
test_data = np.asarray(test_data)
train_data = np.reshape(train_data, (train_data.shape[0], 1, train_data.shape[1]))
test_data = np.reshape(test_data, (test_data.shape[0], 1, test_data.shape[1]))

train_labels = np.asarray(train_labels)

In [24]:
# make, compile and fit the ML model
# input_shape must omit batch size, i.e. : [time steps, features]
model = make_model(input_shape=train_data.shape[1:])
model.compile(optimizer='adam',
              loss=esr_tf,
              metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=10, batch_size=32)

Epoch 1/10
type(y_true)=<class 'tensorflow.python.framework.ops.Tensor'>
y_true=<tf.Tensor 'IteratorGetNext:1' shape=(32, 3) dtype=int64>
type(y_pred)=<class 'tensorflow.python.framework.ops.Tensor'>
y_pred=<tf.Tensor 'sequential_9/dense_9/BiasAdd:0' shape=(32, 1, 3) dtype=float32>


ValueError: in user code:

    File "/var/folders/lf/mzhb79q13fsfdfpwq_qqcrvr0000gn/T/ipykernel_54505/3501082877.py", line 26, in esr_tf  *
        print(f'{y_true.eval()=}')

    ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`
